# Distracted Driving Prediction with Deep Learning
### Sean O'Malley

In [166]:
import time
import numpy as np
#from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam
import pandas as pd
import glob
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import os, sys
from PIL import Image


In [118]:
# Read in easy data
driver_imgs_list = pd.read_csv('driver_imgs_list.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [119]:
driver_imgs_list.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [120]:
sample_submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,img_1000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,img_100000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [121]:
# may have to use a generator???

In [122]:
from PIL import Image

# open and instantiate an image
imagetest = Image.open('imgs/test/img_1.jpg')
print(imagetest)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x1A1E2977B8>


In [123]:
# show the characteristics of an image
from __future__ import print_function
print(imagetest.format, imagetest.size, imagetest.mode)

JPEG (640, 480) RGB


In [124]:
# get format
print(imagetest.format)

JPEG


In [125]:
# get palette
print(imagetest)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x1A1E2977B8>


In [126]:
# show this actual image
imagetest.show()

In [127]:
# save image type
imagetest.save('imagetest.png')

In [128]:
# resize the image
imagetest_new = imagetest.resize((320,240))

In [129]:
# checkout the resize 
imagetest_new.show()

In [130]:
# resize with thumbnail
imagetest_new_thumb = imagetest.thumbnail((320,240))

In [131]:
# get mode
print(imagetest.mode)

RGB


In [133]:
# checkout the thumbnail 
image = Image.open('imagetest.png')
image.show()
#.save('image_thumbnail.jpg')

In [134]:
# cannot instantiate
x = image.thumbnail((8000, 400))
#x.show()

In [159]:
print(sklearn.__version__) 

0.19.1


In [161]:
pip install --upgrade sklearn


The following command must be run outside of the IPython shell:

    $ pip install --upgrade sklearn

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [163]:
X = []
y = []
for j in range(1):
    print('Load folder c{}'.format(j))
    path = os.path.join('imgs', 'train', 'c' + str(j), '*.jpg')
    img_list = glob.glob(path)
    for file in img_list:
        img = Image.open(file).convert('L')
        height, width = img.size
       # thumbnail is a in-place operation
        img = img.resize(( int(height/10), int(width/10) ), Image.ANTIALIAS) #, Image.ANTIALIAS
        pix = np.array(img.getdata()).reshape(img.size[0],img.size[1],1)
        X.append(pix)
        y.append(j)
        
    print("Number of train images: %s" % len(X))

# THIS DOESNT WORK X = np.array(img.getdata()).reshape(img.size[0], img.size[1])  
X = np.array(X, dtype = 'float32') 
y = np.array(y, dtype = 'float32')  

Load folder c0
Number of train images: 2489


In [168]:
#shuffle index rather than sklearn test, train split using np.array random on index
indX = list(range(X.shape[0]))
print(indX[:10])
np.random.shuffle(indX)
print(indX[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1534, 1453, 133, 2258, 2200, 1074, 1617, 2238, 2244, 2444]


In [169]:
Xs=X[indX]
#plt.img(X[0])

In [178]:
#shuffle index rather than sklearn test, train split using np.array random on index
indy = list(range(y.shape[0]))
print(indy[:10])
np.random.shuffle(indy)
print(indy[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2077, 1478, 462, 100, 1522, 1181, 1591, 2460, 596, 790]


In [179]:
ys=y[indy]

In [180]:
print("shape of y:" + str(ys.shape))
print("shape of X:" + str(Xs.shape))

shape of y:(2489,)
shape of X:(2489, 64, 48, 1)


In [ ]:
# now just slice instead of shuffle
#X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8) # shuffle = True

In [181]:
# lets get shape
#X_train.shape

In [ ]:
K.set_image_dim_ordering( 'tf' )

# fix random seed for reproducibility
seed = 123
numpy.random.seed(seed)

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]